In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import tree, ensemble
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import xgboost as xgboost

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/spotifyclassification/data.csv')
data.head(20)

Le but est de prédire si une musique va plaire à un utilisateur en fonction de ses paramètres. Le titre de la chanson ne doit théoriquement pas intervenir dans la prediction, idem pour la durée. Pour l'artiste cela peut peut-être jouer. Nous allons tester le cas avec et sans le nom de l'artiste pour voir son impact sur la prédiction.

Nous supprimons les colonnes inutiles.

In [ ]:
data = data.drop(['song_title', 'duration_ms', 'Unnamed: 0'], axis=1)

## Vérifications de la donnée

In [ ]:
data.info()

In [ ]:
data[data.target == 0].target.count()

In [ ]:
data[data.target == 1].target.count()

Le dataset est assez équilibré

# Prédiction
## Jeu de test

In [ ]:
X = data.drop(['target'], axis=1)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
features = list(X_train.columns)
print(features)

cat_features=['artist']

In [ ]:
y_train.value_counts()

## Catboost

### Catboost avec l'artiste

In [ ]:
model=CatBoostClassifier()
model.fit(X_train,y_train,cat_features=cat_features)
print(classification_report(y_test,model.predict(X_test)))

### Catboost sans l'artiste

In [ ]:
X = data.drop(['target', 'artist'], axis=1)
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model=CatBoostClassifier()
model.fit(X_train,y_train)
print(classification_report(y_test,model.predict(X_test)))

## LogisticRegression

In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_lr = lr.predict(X_test)
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(accuracy_score(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

Avec la regression logistiquen, on obtient un score de 60%

## XGBoost

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

In [ ]:
print(classification_report(y_test, y_xgb))

In [ ]:
xgboost.plot_importance(xgb)

On remarque une légère amélioration avec un arbre de décision

## Forêt

In [ ]:
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(accuracy_score(y_test, y_rf))
print(classification_report(y_test, y_rf))

Il y a une net amélioration avec une forêt aléatoire